In [1]:
# importações
from utils import *
from models import *
from config import OUTPUT_DIR  # Importa a configuração da pasta de dados

# 1. Aplicabilidade do Modelo 2 no Mercado Real

In [2]:
# Stock
stock = ['PETR4']

# Carrega os dados de todas as ações e calcula os indicadores e salva em um dicionario
stock_data = load_stocks_data(stock)

Carregando dados para: PETR4


In [3]:
# Processando o Modelo 2

name_model = model_2
results_list = []

# Usando a Combinação 4: ("Stock Data + IT (IFR + MMS + MME)", stock_data + ifr + mms + mme),
results_list.append(process_stocks_and_save_metrics(stock, stock_data, 4, name_model))

# Combina os resultados em um único DataFrame
results_model_2 = pd.concat(results_list, ignore_index=True)

# Salvar em CSV
results_model_2.to_csv(f'{OUTPUT_DIR}/PETR4/Estudos_IEEE_{name_model.__name__}_02.csv', sep=';', encoding='utf-8-sig')

TypeError: process_stocks_and_save_metrics() takes 3 positional arguments but 4 were given